# Строчные и колоночные БД

## Строчные (традиционные БД)
    Данные храняться как записанные подряд кортежи строк, часто в одном файле
    ```
        ["a", 1, 1.1], ["b", 2, 2.2], ["c", 3, 3.3]
    ```
    Плюсы:
    * Легче реализовать быструю вставку, и изменение записи, транзакционность и т.д.
    Минусы:
    * Даже если запрос затрагивает небольшое кол-во колонок, приходится читать с диска все данные
    * Сложно реализовать векторизацию вычислений

## Колоночные БД
    Каждая колонка храниться как отдельный массив, чаще всего в отдельном файле
    ```
        ["a", "b", "c"],
        [1, 2, 3],
        [1.1, 2.2, 3.3],
    ```
    Плюсы:
    * С диска читаются только колонки действительно нужные для выполнения запроса
    * Векторизация вычислений практечиски "из коробки"
    Минусы:
    * Нужно синхронизировать позицию элементов соответвующих одной строке
    * Как следствие вставка в произвольную позицию довольно сложная с технической точки зрения операция



# DataFrameDBs

https://github.com/waralex/DataFrameDBs.jl

Протопип колоночной БД, написанный на Julia.
Умеет хранить на диске колонки с базовыми типами Julia (все числовые + даты + строки + Union{Missing, T})

Цель написания протопипа прежде всего проверить реализуемость такой БД на Julia и посмотреть на производительность того, что получится

## Кратко об устройстве

* Таблица DataFrameDBs это папка, в которой есть файл `meta.bin` с метаинформацией таблицы(Названия и порядок следвания колонок) и файлы `<номер колонки>.bin` с данными по отдельным колонкам.

* Заголовок файла колонки таблицы выглядит следующим образом:
```
Тип колонки(Строка)
Версия формата(Int64)
Размер блока(в строках)
```

* Данные в файлах колонок разбиты на блоки по n значений (определяется при создании таблицы, по умолчанию 65536 (2^16))

* Каждый блок записывается в файл в следующем виде:
```
Кол-во строк в блоке
Размер данных в блоке в байтах
Размер данных блока после сжатия их с помощью lz2
Данные блока, сжатые lz2
```
* Соответвенно все операции над таблицей происходят следующим образом:

Предположим что нам нужно вывести сумму колонок `a` и `b` при условии что `c == 1`

    1. Определим какие колонки нам нужны для запроса - в нашем случае `a`, `b` и `c`
    2. Создадим в памяти буфферы для этих колонок размером в один блок (т.е. 3 массива в 2^16 элементов каждый)
    3. Прочитаем с диска и разожмем в буфферы данные 1го блока
    4. Выполним на буффере нужные нам операции - т.е. найдем для каких строк `c == 1` и добавим в массив результатов `a+b` из этих строк
    5. Прочитаем с диска в буффер следующий блок и т.д. пока не вычитаем все блоки
    
В результате с одной стороны вне завивисимости от того, сколько строк в таблице для вычислений необходима память только для хранения 1го блока данных (и результата), с другой стороны т.к. блок достаточно большой на нем работает векторизация вычислений




## А теперь как это выглядит с точки зрения пользователя

In [16]:
using OnlineStats

In [17]:
using DataFrameDBs

In [18]:
using Statistics

In [19]:
using UUIDs

In [20]:
using Dates

In [ ]:
ENV["COLUMNS"] = 1000;

In [ ]:
DataFrameDBs.ColumnTypes.serialize(::Type{UUID}) = DataFrameDBs.ColumnTypes.Ast(Symbol("UUID"))
DataFrameDBs.ColumnTypes.deserialize(::Val{Symbol("UUID")}) = UUIDs.UUID

# Табличка ecommerce

Небольшая табличка в 67.5 млн. строк взята отсюда - https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store и сконвертирована в табличку DataFrameDbs

In [21]:
table = open_table("ecommerce")

DFTable path: ecommerce
10×6 DataFrame
│ Row │ column        │ type     │ rows       │ uncompressed size │ compressed size │ compression ratio │
│     │ Symbol        │ String   │ String     │ String            │ String          │ Float64           │
├─────┼───────────────┼──────────┼────────────┼───────────────────┼─────────────────┼───────────────────┤
│ 1   │ event_time    │ DateTime │ 67.5 MRows │ 515.0 MB          │ 22.05 MB        │ 23.36             │
│ 2   │ event_type    │ String   │ 67.5 MRows │ 518.5 MB          │ 28.03 MB        │ 18.5              │
│ 3   │ product_id    │ Int64    │ 67.5 MRows │ 515.0 MB          │ 249.36 MB       │ 2.07              │
│ 4   │ category_id   │ Int64    │ 67.5 MRows │ 515.0 MB          │ 184.1 MB        │ 2.8               │
│ 5   │ category_code │ String   │ 67.5 MRows │ 1.2 GB            │ 288.77 MB       │ 4.26              │
│ 6   │ brand         │ String   │ 67.5 MRows │ 588.95 MB         │ 258.98 MB       │ 2.27              │
│ 7   │

 │ 67.5 MRows │ 1.01 GB           │ 714.7 MB        │ 1.44              │
│ 10  │ Table total   │          │ 67.5 MRows │ 5.8 GB            │ 2.25 GB         │ 2.57              │


In [22]:
head(table)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
,DateTime,String,Int64,Int64,String,String,Float64,Int64,UUID
1,2019-11-01T00:00:00,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,"UUID(""4d3b30da-a5e4-49df-b1a8-ba5943f1dd33"")"
2,2019-11-01T00:00:00,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,"UUID(""8e5f4f83-366c-4f70-860e-ca7417414283"")"
3,2019-11-01T00:00:01,view,17302664,2053013553853497655,,creed,28.31,561587266,"UUID(""755422e7-9040-477b-9bd2-6a6e8fd97387"")"
4,2019-11-01T00:00:01,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,"UUID(""3bfb58cd-7892-48cc-8020-2f17e6de6e7f"")"
5,2019-11-01T00:00:01,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,"UUID(""313628f1-68b8-460d-84f6-cec7a8796ef2"")"
6,2019-11-01T00:00:01,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,"UUID(""816a59f3-f5ae-4ccd-9b23-82aa8c23d33c"")"
7,2019-11-01T00:00:01,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,"UUID(""df8184cc-3694-4549-8c8c-6b5171877376"")"
8,2019-11-01T00:00:02,view,15900065,2053013558190408249,,rondell,30.86,518574284,"UUID(""5e6ef132-4d7c-4730-8c7f-85aa4082588f"")"
9,2019-11-01T00:00:02,view,12708937,2053013553559896355,,michelin,72.72,532364121,"UUID(""0a899268-31eb-46de-898d-09b2da950b24"")"


* включим отоброжение прогресса выполенния запросов

In [23]:
turnon_progress!(table);

## DFColumn - lazy колонка - итератор к данным, лежащим на диске

In [24]:
table.event_type

DFColumn{String}

In [25]:
length(table.event_type)

Time: 0:00:0.2924 read: 67.5 MRows (230.89 MRows/sec))


67501979

In [26]:
unique(table.event_type)

Time: 0:00:20.6292 read: 67.5 MRows (3.27 MRows/sec))0:00:1.1076 read: 2.82 MRows (2.54 MRows/sec)0:00:1.8820 read: 5.51 MRows (2.93 MRows/sec)0:00:2.6472 read: 8.06 MRows (3.05 MRows/sec)0:00:3.3064 read: 10.22 MRows (3.09 MRows/sec)0:00:4.0617 read: 12.78 MRows (3.15 MRows/sec)0:00:4.8561 read: 15.47 MRows (3.18 MRows/sec)0:00:5.7405 read: 18.48 MRows (3.22 MRows/sec)0:00:6.3943 read: 20.71 MRows (3.24 MRows/sec)0:00:7.8258 read: 25.62 MRows (3.27 MRows/sec)0:00:8.6028 read: 28.31 MRows (3.29 MRows/sec)0:00:9.2655 read: 30.54 MRows (3.3 MRows/sec)0:00:10.6924 read: 35.45 MRows (3.32 MRows/sec)0:00:12.2716 read: 40.63 MRows (3.31 MRows/sec)0:00:13.0762 read: 43.32 MRows (3.31 MRows/sec)0:00:13.8365 read: 45.81 MRows (3.31 MRows/sec)0:00:14.5108 read: 48.04 MRows (3.31 MRows/sec)0:00:15.0685 read: 49.74 MRows (3.3 MRows/sec)0:00:15.8461 read: 52.36 MRows (3.3 MRows/sec)0:00:17.1952 read: 56.82 MRows (3.3 MRows/sec)0:00:17.8386 read: 58.79 MRows (3.3 MRows/sec)0:00:18.3793 read: 60.49 M

3-element Vector{String}:
 "view"
 "cart"
 "purchase"

### Индексация DFColumn - это тоже DFColumn. Индексация применяется так же в lazy режиме при материалицазии колонки

In [27]:
ev_10 = table.event_type[1:100:30_000_000]

DFColumn{String}

In [28]:
size(ev_10)

Time: 0:00:0.7207 read: 30.02 MRows (41.65 MRows/sec)


(300000,)

In [29]:
ev_10[1:10]

DFColumn{String}

In [30]:
materialize(ev_10[1:10])

Time: 0:00:0.0002 read: 65.54 KRows (271.86 MRows/sec)


10-element Vector{String}:
 "view"
 "view"
 "view"
 "view"
 "view"
 "view"
 "view"
 "view"
 "view"
 "view"

### Любые  broadcast операции над DFColumn - тоже DFColumn. Операции так же выполняются в lazy режиме

In [31]:
purchase_condition = table.event_type .== "purchase"

DFColumn{Bool}

In [32]:
count(purchase_condition)

Time: 0:00:12.5987 read: 67.5 MRows (5.36 MRows/sec))0:00:0.7141 read: 3.87 MRows (5.41 MRows/sec)0:00:1.4631 read: 8.13 MRows (5.55 MRows/sec)0:00:3.0360 read: 17.17 MRows (5.66 MRows/sec)0:00:3.7761 read: 21.43 MRows (5.68 MRows/sec)0:00:5.3482 read: 29.95 MRows (5.6 MRows/sec)0:00:6.1821 read: 34.34 MRows (5.55 MRows/sec)0:00:6.9063 read: 38.27 MRows (5.54 MRows/sec)0:00:8.8308 read: 48.23 MRows (5.46 MRows/sec)0:00:10.0261 read: 54.33 MRows (5.42 MRows/sec)0:00:11.1034 read: 59.83 MRows (5.39 MRows/sec)


916939

In [33]:
tmp = table.price .* 2 ./ table.user_id

DFColumn{Float64}

In [34]:
materialize(tmp[1:5])

Time: 0:00:0.0004 read: 65.54 KRows (147.26 MRows/sec)


5-element Vector{Float64}:
 1.8807169168138991e-6
 1.1070755018140636e-6
 1.0082137439348562e-7
 2.751939109613261e-6
 6.558747728172019e-7

In [35]:
f(brand, price) = brand == "xiaomi" ? price * 1.2 : price * 0.8

new_price = f.(table.brand, table.price)

DFColumn{Float64}

In [36]:
materialize(new_price[1:10])

Time: 0:00:0.0003 read: 65.54 KRows (253.46 MRows/sec)


10-element Vector{Float64}:
 586.884
 234.92
  22.648
 570.296
 219.924
 288.072
 411.64799999999997
  24.688000000000002
  58.176
 585.6560000000001

### DFColumn{Bool} может использоваться для индексации колонки

In [37]:
purchase_prices = table.price[table.event_type .== "purchase"]

DFColumn{Float64}

In [38]:
length(purchase_prices)

Time: 0:00:3.5548 read: 67.5 MRows (18.99 MRows/sec))0:00:2.5408 read: 48.43 MRows (19.06 MRows/sec)


916939

In [39]:
mean(purchase_prices)

Time: 0:00:4.9554 read: 67.5 MRows (13.62 MRows/sec))0:00:3.3655 read: 47.05 MRows (13.98 MRows/sec)


300.12344387126365

## DFView - вью таблицы.

Задается правилами `selection` (фильтрация и/или индексация) и `projection` - набор колонок, которые входят во вьюху
Так же как и DFColumn реальное чтение данных и выполнение вычислений происходит в момент материализации DFView
Синтаксис работы с DFView максимально приближен к синтаксису DataFrame


### DFView полной таблицы

In [40]:
full_view = table[:,:]

View of table ecommerce
Projection: event_time=>col(event_time)::DateTime; event_type=>col(event_type)::String; product_id=>col(product_id)::Int64; category_id=>col(category_id)::Int64; category_code=>col(category_code)::String; brand=>col(brand)::String; price=>col(price)::Float64; user_id=>col(user_id)::Int64; user_session=>col(user_session)::UUID
Selection: 


In [ ]:
full_view.selection

In [41]:
head(full_view)

Time: 0:00:0.0003 read: 65.54 KRows (228.96 MRows/sec)
Time: 0:00:0.0005 read: 65.54 KRows (122.02 MRows/sec)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
,DateTime,String,Int64,Int64,String,String,Float64,Int64,UUID
1,2019-11-01T00:00:00,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,"UUID(""4d3b30da-a5e4-49df-b1a8-ba5943f1dd33"")"
2,2019-11-01T00:00:00,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,"UUID(""8e5f4f83-366c-4f70-860e-ca7417414283"")"
3,2019-11-01T00:00:01,view,17302664,2053013553853497655,,creed,28.31,561587266,"UUID(""755422e7-9040-477b-9bd2-6a6e8fd97387"")"
4,2019-11-01T00:00:01,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,"UUID(""3bfb58cd-7892-48cc-8020-2f17e6de6e7f"")"
5,2019-11-01T00:00:01,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,"UUID(""313628f1-68b8-460d-84f6-cec7a8796ef2"")"
6,2019-11-01T00:00:01,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,"UUID(""816a59f3-f5ae-4ccd-9b23-82aa8c23d33c"")"
7,2019-11-01T00:00:01,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,"UUID(""df8184cc-3694-4549-8c8c-6b5171877376"")"
8,2019-11-01T00:00:02,view,15900065,2053013558190408249,,rondell,30.86,518574284,"UUID(""5e6ef132-4d7c-4730-8c7f-85aa4082588f"")"
9,2019-11-01T00:00:02,view,12708937,2053013553559896355,,michelin,72.72,532364121,"UUID(""0a899268-31eb-46de-898d-09b2da950b24"")"


### DFView с фильтрацией и отбором колонок

In [42]:
purchases = table[table.event_type .== "purchase", [:price, :user_id, :product_id, :brand]]

View of table ecommerce
Projection: price=>col(price)::Float64; user_id=>col(user_id)::Int64; product_id=>col(product_id)::Int64; brand=>col(brand)::String
Selection: ==(col(event_type)::String, Base.RefValue{String}("purchase"))::Bool


In [ ]:
purchases.selection

In [43]:
size(purchases)

Time: 0:00:3.6607 read: 67.5 MRows (18.44 MRows/sec))


(916939, 4)

In [44]:
expensive_purchases = purchases[purchases.price .>= 2000, :]

View of table ecommerce
Projection: price=>col(price)::Float64; user_id=>col(user_id)::Int64; product_id=>col(product_id)::Int64; brand=>col(brand)::String
Selection: &(==(col(event_type)::String, Base.RefValue{String}("purchase"))::Bool, >=(col(price)::Float64, 2000)::Bool)::Bool


In [45]:
size(expensive_purchases)

Time: 0:00:5.9688 read: 67.5 MRows (11.31 MRows/sec))0:00:1.6348 read: 18.15 MRows (11.1 MRows/sec)0:00:4.2953 read: 47.97 MRows (11.17 MRows/sec)0:00:5.6380 read: 63.7 MRows (11.3 MRows/sec)


(1459, 4)

### Схлопывание условий в selection

In [46]:
println(expensive_purchases.selection)
expensive_purchases2 = purchases[1:1000, :][purchases[1:1000, :].price .>= 2000, :]
println(expensive_purchases2.selection)
println(expensive_purchases2[1:100, :].selection)
println(expensive_purchases2[1:100:100000, :][1:5, :].selection)

Selection: &(==(col(event_type)::String, Base.RefValue{String}("purchase"))::Bool, >=(col(price)::Float64, 2000)::Bool)::Bool
Selection: ==(col(event_type)::String, Base.RefValue{String}("purchase"))::Bool |> 1:1000 |> >=(col(price)::Float64, 2000)::Bool
Selection: ==(col(event_type)::String, Base.RefValue{String}("purchase"))::Bool |> 1:1000 |> >=(col(price)::Float64, 2000)::Bool |> 1:100
Selection: ==(col(event_type)::String, Base.RefValue{String}("purchase"))::Bool |> 1:1000 |> >=(col(price)::Float64, 2000)::Bool |> 1:100:401


### Проиводные колонки

In [47]:
convert_price(brand, price) = brand == "xiaomi" ? price * 1.2 : price * 0.8

convert_price (generic function with 1 method)

In [ ]:
[
        :brand => :brand,
        
    ]

In [48]:
price_changes_table = table[:,
    (
        brand = :brand,
        old_price = :price,
        new_price = (:brand, :price) => convert_price        
        
    )
]


View of table ecommerce
Projection: brand=>col(brand)::String; old_price=>col(price)::Float64; new_price=>convert_price(col(brand)::String, col(price)::Float64)::Float64
Selection: 


In [49]:
head(price_changes_table)

Time: 0:00:0.0013 read: 65.54 KRows (52.17 MRows/sec)
Time: 0:00:0.0957 read: 65.54 KRows (684.47 KRows/sec)


,brand,old_price,new_price
,String,Float64,Float64
1,xiaomi,489.07,586.884
2,janome,293.65,234.92
3,creed,28.31,22.648
4,lg,712.87,570.296
5,xiaomi,183.27,219.924
6,hp,360.09,288.072
7,hp,514.56,411.648
8,rondell,30.86,24.688
9,michelin,72.72,58.176


In [50]:
price_changes_table = price_changes_table[:,
    (
        brand = :brand,
        old_price = :old_price,
        new_price = :new_price,
        price_change = (:old_price, :new_price) => (o, n) -> n - o        
        
    )
]


View of table ecommerce
Projection: brand=>col(brand)::String; old_price=>col(price)::Float64; new_price=>convert_price(col(brand)::String, col(price)::Float64)::Float64; price_change=>#1(col(price)::Float64, convert_price(col(brand)::String, col(price)::Float64)::Float64)::Float64
Selection: 


In [51]:
head(price_changes_table)

Time: 0:00:0.0003 read: 65.54 KRows (208.85 MRows/sec)
Time: 0:00:0.0473 read: 65.54 KRows (1.39 MRows/sec)


,brand,old_price,new_price,price_change
,String,Float64,Float64,Float64
1,xiaomi,489.07,586.884,97.814
2,janome,293.65,234.92,-58.73
3,creed,28.31,22.648,-5.662
4,lg,712.87,570.296,-142.574
5,xiaomi,183.27,219.924,36.654
6,hp,360.09,288.072,-72.018
7,hp,514.56,411.648,-102.912
8,rondell,30.86,24.688,-6.172
9,michelin,72.72,58.176,-14.544


In [52]:
sum(price_changes_table.price_change)

Time: 0:00:21.4617 read: 67.5 MRows (3.15 MRows/sec))0:00:0.6343 read: 1.9 MRows (3.0 MRows/sec)0:00:1.1485 read: 3.54 MRows (3.08 MRows/sec)0:00:1.8146 read: 5.44 MRows (3.0 MRows/sec)0:00:2.4726 read: 7.34 MRows (2.97 MRows/sec)0:00:3.1162 read: 9.24 MRows (2.97 MRows/sec)0:00:3.7383 read: 11.14 MRows (2.98 MRows/sec)0:00:4.3709 read: 13.11 MRows (3.0 MRows/sec)0:00:5.0042 read: 15.07 MRows (3.01 MRows/sec)0:00:5.6389 read: 17.04 MRows (3.02 MRows/sec)0:00:6.2707 read: 19.01 MRows (3.03 MRows/sec)0:00:6.8007 read: 20.64 MRows (3.04 MRows/sec)0:00:7.3365 read: 22.28 MRows (3.04 MRows/sec)0:00:7.8556 read: 23.92 MRows (3.05 MRows/sec)0:00:8.5025 read: 25.89 MRows (3.04 MRows/sec)0:00:9.0399 read: 27.53 MRows (3.04 MRows/sec)0:00:9.6695 read: 29.49 MRows (3.05 MRows/sec)0:00:10.3060 read: 31.46 MRows (3.05 MRows/sec)0:00:10.9523 read: 33.42 MRows (3.05 MRows/sec)0:00:11.4724 read: 35.06 MRows (3.06 MRows/sec)0:00:12.1502 read: 37.22 MRows (3.06 MRows/sec)0:00:12.9301 read: 39.78 MRows (

-3.5703025117562428e9

## Аггрегация

Пока только с помощью OnlineStats что не очень эффективно, но работает

In [53]:
fit!(GroupBy(String, Series(Mean(), Extrema())), zip(purchases.brand, purchases.price))

Time: 0:00:11.0807 read: 67.5 MRows (6.09 MRows/sec)d:  (66.98 MRows6.1 MRows/sec ()6.1 MRows/sec)))22m) read: 0:00:2.371913.11 MRows read:  (13.11 MRows5.5 MRows/sec ()5.53 MRows/sec) read: 0:00:3.185418.15 MRows read:  (18.15 MRows5.68 MRows/sec ()5.7 MRows/sec) read: 0:00:4.203624.64 MRows read:  (24.64 MRows5.85 MRows/sec ()5.86 MRows/sec) read: 0:00:7.555447.19 MRows read:  (47.19 MRows6.24 MRows/sec ()6.25 MRows/sec)


GroupBy: String => Series
├─ ""
│  └─ Series
│     ├─ Mean: n=73273 | value=150.475
│     └─ Extrema: n=73273 | value=(min = 0.79, max = 2574.07, nmin = 3, nmax = 7)
├─ "xiaomi"
│  └─ Series
│     ├─ Mean: n=68292 | value=164.878
│     └─ Extrema: n=68292 | value=(min = 1.29, max = 2033.51, nmin = 2, nmax = 1)
├─ "samsung"
│  └─ Series
│     ├─ Mean: n=200027 | value=274.312
│     └─ Extrema: n=200027 | value=(min = 1.26, max = 2574.04, nmin = 23, nmax = 6)
├─ "lucente"
│  └─ Series
│     ├─ Mean: n=14559 | value=242.293
│     └─ Extrema: n=14559 | value=(min = 11.33, max = 971.97, nmin = 1, nmax = 48)
├─ "nakamichi"
│  └─ Series
│     ├─ Mean: n=81 | value=83.7544
│     └─ Extrema: n=81 | value=(min = 28.83, max = 123.56, nmin = 1, nmax = 13)
├─ "magnetta"
│  └─ Series
│     ├─ Mean: n=250 | value=239.141
│     └─ Extrema: n=250 | value=(min = 20.59, max = 782.39, nmin = 1, nmax = 1)
├─ "cortland"
│  └─ Series
│     ├─ Mean: n=190 | value=98.1393
│     └─ Extrema: n=190 | value=(min =

In [ ]:
mean(purchases.price[purchases.brand .== "xiaomi"])

# Как все это можно развивать

## Типы данных
    * массивы
    * CategiricalArrays
    * NamedTuple
  
## Аггрегация
    Тут много всего, от простой аггрегации в памяти, до аггрегации с использованием временных файлов и предварительно вычисленных индексов

## Индексы
    * Классические индексы малополезны, но можно, например, хранить Bloom фильтр для каждого блока - это должно хорошо ускорить фильтрацию

## Join
    * Довольно тяжелы в реализации, тем не менее вполне реальны
    
## Всякие разности
    * Пересортировка таблиц на диске без поднятия их полностью в память
    * Вставка в произвольную позицию / изменение записей
    * Всякие "индексы" для групировок/джойнов, возможно автоматические. Т.е. индекс создается с каким-то TTL при джойне

## Инфраструктура
    * Сущность "База данных"
    * Фоновые процессы - всякий дорасчет индексов, предагреггации и т.д.
    * Сетевой интерфейс
    * Шардинг, распределенные вычисления и т.д.
    
# Главный вопрос - а надо ли? )
